# Часть 2. Генерация голоса


## Установка зависимостей и инициализация модели

In [ ]:
#@title Выберете язык (только 1):

english=True#@param {type:"boolean"}
french = False#@param {type:"boolean"}
german = False#@param {type:"boolean"}

In [ ]:
#@title Установка модели для озвучивания:

!python3 -m venv larynx_venv
!source larynx_venv/bin/activate

!pip3 install --upgrade pip
!pip3 install --upgrade wheel setuptools

#не обращать внимания на restart runtime

In [ ]:
!pip3 install -f 'https://synesthesiam.github.io/prebuilt-apps/' larynx
!python3 -m larynx

#не обращать внимания на restart runtime

In [ ]:
!mkdir voices #создаём папку, куда будем складывать аудио файлы

## Текст из ноутбука со сказками

In [ ]:
data = "One sunny day, the four wives of the old British consul, the vicar, and a young couple head to the seaside country of Oaxham to attend the wedding of a young man. ^ I think the three old ladies are all of married social standing ^ ." 
#!!!Вставить переменную sentence из ноутбука "Фильм_часть1"!!!

In [ ]:
data = data.split('.')[:-1] #разбиваем по фразам для дальнейшего создания таймодов

In [ ]:
data

In [ ]:
len(data)

## Генерация речи 

In [ ]:
import os

In [ ]:
#@title Выбор голосов для озвучивания
if english:
  narr = 'cmu_eey' #расскащик
  persMal = 'cmu_bdl' #мужской персонаж
  persFem = 'blizzard_lessac' #женский персонаж

elif french:
  narr = 'gilles_le_blanc'
  persMal = 'tom'
  persFem = 'siwis'

elif german:
  narr = 'eva_k'
  persMal = 'pavoque'
  persFem = 'kerstin'

In [ ]:
if english:
  he = 'he'
  she = 'she'

elif french:
  he = 'il'
  she = 'elle'

elif german:
  he = 'Er'
  she = 'Sie'

In [ ]:
for j in range(len(data)):
  isPersMal = False
  isPersFem = False

  words = data[j].split(' ')
  a = 0

  for i in range(len(words)):
    if words[i] == '^':
      a+=1
      if a==2 and words[i+1] == he: #поиск указаний на пол персонажа
        isPersMal = True
      elif a==2 and words[i+1] == she:
        isPersFem = True  
      elif a==2:
        isPersMal = True #Поменять на isPersFem если хотите женский голос персонажей по умолчанию
      

  text=data[j]
  text+='.'
  name = f'/content/voices/{j:03}.wav' 
  if isPersMal:
    command = os.popen('larynx -v '+ persMal +' "'+text+'" -q "high" > '+name) #озвучивание(по фразовое)
  elif isPersFem:
    command = os.popen('larynx -v ' +persFem+ ' "'+text+'" -q "high" > '+name)
  else:
    command = os.popen('larynx -v ' +narr+ ' "'+text+'" -q "high" > '+name)
  execute = command.read
  command.close()

In [ ]:
import soundfile as sf

In [ ]:
#@title Генерация таймкодов
timecodes = []
duration = 0
for j in range(len(data)):
  file = sf.SoundFile(f'/content/voices/{j:03}.wav')
  duration += len(file)/file.samplerate
  timecodes+=[{'start':duration,'text':data[j]}]
print(timecodes)

In [ ]:
timecodes

In [ ]:
import wave

In [ ]:
#@title Обьединение голосовых файлов в один

outfile = 'voice' #@param {type:'string'}
outfile += '.wav' 
data1 = []
for j in range(len(data)):
  w = wave.open(f'/content/voices/{j:03}.wav', 'rb')
  data1.append([w.getparams(), w.readframes(w.getnframes())])
  w.close()

output = wave.open(outfile, 'wb')
output.setparams(data1[0][0])
for j in range(len(data)):
  output.writeframes(data1[j][1])
output.close

In [ ]:
import IPython

IPython.display.Audio('/content/voice.wav')

#Пожалуйста, сохраните файл voice.wav и переменную timecodes чтобы использовать их в "Фильм_часть2-2" и "Фильм_часть3" )